In [1]:
import numpy as np
import netCDF4 as nc
import glob

In [2]:
file_path = '/gws/nopw/j04/tone_ico_gws/cloudnet/neumayer/radar/'

file_list = sorted(glob.glob('/gws/nopw/j04/tone_ico_gws/cloudnet/neumayer/radar/*.nc'))  # Adjust path


In [4]:
# Example dimensions (adjust as needed)
n_days = 365
uniform_time_dim = 2879     # e.g., 5-minute resolution over 24 hours

file_path = '/gws/nopw/j04/tone_ico_gws/cloudnet/neumayer/radar'

with nc.Dataset('snowfall_radar.nc', 'w', format='NETCDF4') as ds:
    ds.createDimension('day', n_days)
    ds.createDimension('time', uniform_time_dim)

    lower_var = ds.createVariable('lower_bound', 'f4', ('day', 'time'), zlib=True)
    upper_var = ds.createVariable('upper_bound', 'f4', ('day', 'time'), zlib=True)

    lower_var.units = 'mm/hr'
    upper_var.units = 'mm/hr'
    ds.description = 'Daily snowfall bounds at surface point from radar reflectivity'

    file_path = '/gws/nopw/j04/tone_ico_gws/cloudnet/neumayer/radar/'

    file_list = sorted(glob.glob('/gws/nopw/j04/tone_ico_gws/cloudnet/neumayer/radar/*.nc'))  # Adjust path

    for day_idx, file in enumerate(file_list):
        with nc.Dataset(file) as f:
            dbz = np.array(f.variables['Zh'][:, 0])
            dbz[dbz > 1000] = np.nan
            z = 10 ** (dbz / 10)

            aLower = 11
            aHigher = 63
            b = 1.1

            native_time = dbz.shape[0]

            precipLower_native = (z / aHigher) ** (1 / b)
            precipHigher_native = (z / aLower) ** (1 / b)


            lower_resampled = np.interp(np.arange(0, uniform_time_dim), np.arange(0, native_time), precipLower_native)
            upper_resampled = np.interp(np.arange(0, uniform_time_dim), np.arange(0, native_time), precipHigher_native)


            lower_var[day_idx, :] = lower_resampled
            upper_var[day_idx, :] = upper_resampled





KeyboardInterrupt: 